# API Tutorial

This document aims to give a brief tutorial regarding the Python API provided to interact with the Docker environment.

The modules and functions are located in the `utilities` folder of the Starter-Kit repository.

## Running competition_executor with the command line

The `competition_executor.py` file can be run with the following syntax:

`python utilities/competition_executor.py <path_input> <path_output> <n_seconds>`

The three arguments are as follows:
* `path_input`: absolute path to the submission_input directory
* `path_output`: absolute path to an output directory of your choice. 
* `n_seconds`: Number of seconds the simulation is going to run before the program quits.

## Importing competition_executor as a module

For users who want to import the `competition_executor` module to use in their own scripts / Jupyter notebooks, the cells below describe the steps to follow.

In [1]:
# Adding the module to the path for future import
path_competition_executor = "/Users/vgolfi/Documents/GitHub/Uber-Prize-Starter-Kit/utilities"
import sys
import docker
sys.path.append(path_competition_executor)
from competition_executor import CompetitionContainerExecutor

First, a **`CompetitionContainorExecutor` object** must be created based on an input location and an output location. <br>(Note that the definition of those two arguments is not mandatory. If not, the simulation will use the directories defined at instantiation of the `CompetitionContainorExecutor` object.)

In [2]:
path_input = r"/Users/vgolfi/Documents/GitHub/Uber-Prize-Starter-Kit/submission-inputs"
path_ouput = r"/Users/vgolfi/Documents/GitHub/Uber-Prize-Starter-Kit/output"

ex = CompetitionContainerExecutor(input_root=path_input,
                                  output_root=path_ouput)

Then and before running a simulation, a **container ID** has to be defined.

In [3]:
CONTAINER_ID = 'uber1'

Once the container id created, a simulation can be run with the following arguments:
* `submission_id` / `container_id`: identifier of the simulation instance (will become the container name)
* `submission_output_root`: the (absolute) path to locate simulation outputs
* `submission_input_root`: (absolute) path where simulation inputs are located
* `scenario_name`: which of the available scenarios will be run in the container
* `sample_size`: available samples size (scenario dependent, see documentation).
* `num_iterations`: number of iterations to run BEAM simulation engine.


Pay attention to the fact that two docker containers with the same `CONTAINER_ID` cannot coexist. Trying to run a simulation with a `CONTAINER_ID` that already exists will raise an error.

You can use either a try statement to remove an already existing container before running a new simulation:

In [4]:
try:
    uber = ex.client.containers.get(CONTAINER_ID)
    uber.stop()
    uber.remove()
    print("Found container from previous run, removing and creating new sim container...")
except docker.errors.NotFound:
    print("Creating new simulation container...")

Found container from previous run, removing and creating new sim container...


Next, **run the simulation** with the following method:

In [5]:
ex.run_simulation(CONTAINER_ID, num_iterations=1, num_cpus=10)

Note that a dictionary indicating which containers IDs already exist can be accessed.

In [6]:
for key, value in ex.containers.items():
    print("Container ID : {0}".format(key))

Container ID : uber1


To see the **logs** of a specific simulation ID, write the following method:

In [10]:
logs = ex.output_simulation_logs(CONTAINER_ID)

06:28:44.881 DEBUG beam.utils.BeamConfigUtils$ - Loading beam config from fixed-data/siouxfalls/siouxfalls-1k.conf.
06:28:46.026 DEBUG b.sim.config.MatSimBeamConfigBuilder - Couldn't instantiate MatsimParameters  'org.matsim.run.OldToNewPlanCalcScoreConfigGroup.ModeParams'. It doesn't have default public constructor.Falling back to setAccessible(). Cause : Class beam.sim.config.MatSimBeamConfigBuilder$$anonfun$1 can not access a member of class org.matsim.run.OldToNewPlanCalcScoreConfigGroup$ModeParams with modifiers ""
06:28:46.048 DEBUG b.sim.config.MatSimBeamConfigBuilder - Couldn't instantiate MatsimParameters  'org.matsim.core.config.groups.PlanCalcScoreConfigGroup.ModeParams'. It doesn't have default public constructor.Falling back to setAccessible(). Cause : Class beam.sim.config.MatSimBeamConfigBuilder$$anonfun$1 can not access a member of class org.matsim.core.config.groups.PlanCalcScoreConfigGroup$ModeParams with modifiers ""
06:28:46.176 DEBUG beam.utils.FileUtils$ - Beam ou

Submissions run for a certain amount of time. If you interrupt it before the end, you will not get any outputs. You can **check if a submission is finished** with the following method:

In [11]:
ex.check_if_submission_complete(CONTAINER_ID)

True

When a simulation run is done, you can import the **results** of the specified simulation name.

In [12]:
scores, stats = ex.get_submission_scores_and_stats(CONTAINER_ID)

The **scores** and **statistics** are stored in pandas dataframes which contains the information described [here](https://github.com/vgolfier/Uber-Prize-Starter-Kit/blob/master/docs/Understanding_the_outputs_and_the%20scoring_function.md).

In [13]:
scores

,Budget: Operational Costs (Fixed)_Weight,Budget: Operational Costs (Fuel)_Weight,"Budget: Operational Costs (Variable, Hourly)_Weight",Budget: Subsidies Paid_Weight,Budget: Subsidies Unpaid_Weight,Congestion: Vehicle Hours Delay_Weight,Congestion: Vehicle Miles Traveled_Weight,Level of Service: Agent Hours on Crowded Transit_Weight,Level of Service: Travel Expenditure_Weight,Submission Score_Weight,...,Budget: Operational Costs (Fixed)_Weighted Score,Budget: Operational Costs (Fuel)_Weighted Score,"Budget: Operational Costs (Variable, Hourly)_Weighted Score",Budget: Subsidies Paid_Weighted Score,Budget: Subsidies Unpaid_Weighted Score,Congestion: Vehicle Hours Delay_Weighted Score,Congestion: Vehicle Miles Traveled_Weighted Score,Level of Service: Agent Hours on Crowded Transit_Weighted Score,Level of Service: Travel Expenditure_Weighted Score,Submission Score_Weighted Score
0,0.001,0.001,0.001,0.001,0.001,0.01,0.0001,10.0,0.0001,0.0,...,0.0,4.547474e-16,0.0,0.0,0.0,-16.152006,-0.180588,-94.691667,-0.057419,-111.081679


In [14]:
stats

,Iteration,agentHoursOnCrowdedTransit,fuelConsumedInMJ_Diesel,fuelConsumedInMJ_Food,fuelConsumedInMJ_Gasoline,numberOfVehicles_BODY-TYPE-DEFAULT,numberOfVehicles_BUS-DEFAULT,numberOfVehicles_CAR-TYPE-DEFAULT,personTravelTime_car,personTravelTime_drive_transit,...,totalSubsidy_walk_transit,totalTravelTime,totalVehicleDelay,vehicleHoursTraveled_BODY-TYPE-DEFAULT,vehicleHoursTraveled_BUS-DEFAULT,vehicleHoursTraveled_CAR-TYPE-DEFAULT,vehicleMilesTraveled_BODY-TYPE-DEFAULT,vehicleMilesTraveled_BUS-DEFAULT,vehicleMilesTraveled_CAR-TYPE-DEFAULT,vehicleMilesTraveled_total
0,0,9.094444,130444.966235,45.973342,23397.588354,1003.0,255.0,996.0,18129.583333,2448.433333,...,0.0,8054.445833,8139.478333,186.286944,156.916667,332.384167,538.990758,4043.033927,3976.659436,8558.684121
1,1,9.469167,130444.966235,49.702440,23177.005382,1003.0,255.0,1124.0,35653.000000,2789.233333,...,0.0,8036.155278,8157.020556,201.402500,156.916667,648.013333,582.710652,4043.033927,3939.169104,8564.913682


At the end of any run and to avoid any conflicts between `CONTAINER_ID`s (particularly if you want to reuse names), it is advised to **clean up containers** calling the following method:

In [15]:
ex.stop_all_simulations()